In [41]:
import numpy as np
import sys
import os

# Import module tự viết
sys.path.append(os.path.abspath(".."))
from src.data_processing import load_csv, detect_numeric_columns

# 01. Load dữ liệu 
file_path = "../data/raw/BankChurners.csv"
header, data_raw = load_csv(file_path)

print("Header shape:", len(header))
print("Data shape:", data_raw.shape)

# Chuyển header thành array
header_arr = np.array(header)

Header shape: 23
Data shape: (10127, 23)


In [42]:
# 02. Xử lý giá trị "Unknown" (Imputation bằng Mode)

data = data_raw.copy()
# Tìm các cột có chứa "Unknown"
has_unknown = np.any((data == "Unknown"), axis=0)
cols_with_unknown = header_arr[has_unknown]
print(f"Các cột xử lý Unknown: {cols_with_unknown}")
for col in cols_with_unknown:
    col_idx = np.where(header_arr == col)[0][0]
    col_data = data[:, col_idx]
    # Lọc ra các giá trị hợp lệ (không phải Unknown)
    valid_vals = col_data[col_data != "Unknown"]  
    # Tìm Mode (giá trị xuất hiện nhiều nhất)
    vals, counts = np.unique(valid_vals, return_counts=True)
    mode_val = vals[np.argmax(counts)] 
    # Thay thế "Unknown" bằng Mode
    data[data[:, col_idx] == "Unknown", col_idx] = mode_val


Các cột xử lý Unknown: ['Education_Level' 'Marital_Status' 'Income_Category']


#### Bước này thay thế tất cả "Unknown" bằng (mode) giá trị xuất hiện nhiều nhất trong cột tương ứng.

In [ ]:
# 03. Loại bỏ các cột không dùng
remove_cols = [
    "CLIENTNUM",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
]
keep_mask = ~np.isin(header_arr, remove_cols)
data = data[:, keep_mask]
header_arr = header_arr[keep_mask]
print("Shape sau khi loại bỏ cột:", data.shape)


Shape sau khi loại bỏ cột: (10127, 20)


In [49]:
# 04. Feature Engineering (Tạo đặc trưng mới)

def get_col(name):
    idx = np.where(header_arr == name)[0][0]
    return data[:, idx].astype(float)
# Tạo thêm biến 1: Avg_Trans_Amt_Per_Transaction (Giá trị trung bình mỗi lần giao dịch)
# Công thức: Total_Trans_Amt / Total_Trans_Ct
amt = get_col('Total_Trans_Amt')
count = get_col('Total_Trans_Ct')
# Tránh chia cho 0 bằng cách cộng 1e-9 hoặc dùng np.where
avg_trans = np.divide(amt, count, out=np.zeros_like(amt), where=count!=0)
# Tạo thêm biến 2: Credit_Utilization_Rate _Calc (Tỷ lệ sử dụng tín dụng)
# Công thức: Total_Revolving_Bal / Credit_Limit
bal = get_col('Total_Revolving_Bal')
limit = get_col('Credit_Limit')
util_rate = np.divide(bal, limit, out=np.zeros_like(bal), where=limit!=0)
# Thêm 2 cột mới vào data
new_features = np.column_stack((avg_trans, util_rate)).astype(str) # Chuyển về str để ghép vào data chính
data = np.hstack((data, new_features))
# Cập nhật header
header_arr = np.append(header_arr, ['Avg_Trans_Amt_Per_Transaction', 'Credit_Utilization_Rate_Calc'])
print("Đã thêm 2 Features mới. Shape:", data.shape)

Đã thêm 2 Features mới. Shape: (10127, 24)


In [50]:
# 05. Tách Target và Phân loại cột

target_col = "Attrition_Flag"
target_idx = np.where(header_arr == target_col)[0][0]
# Tạo y (1: Attrited, 0: Existing)
y = (data[:, target_idx] == "Attrited Customer").astype(int)
# Tạo X (bỏ cột target)
X_raw = np.delete(data, target_idx, axis=1)
X_header = np.delete(header_arr, target_idx)
# Xác định lại cột số và cột phân loại tự động
cat_cols_manual = ["Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"]
# Lấy index của các cột numeric (những cột ko nằm trong list cat)
num_mask = ~np.isin(X_header, cat_cols_manual)
cat_mask = np.isin(X_header, cat_cols_manual)
# Tách dữ liệu ra 2 phần riêng biệt
X_num = X_raw[:, num_mask].astype(float)
X_cat = X_raw[:, cat_mask]
num_header = X_header[num_mask]
cat_header = X_header[cat_mask]
print("Số cột dữ liệu là só:", len(num_header))
print("Số cột dữ liệu là phân loại:", len(cat_header))

Số cột dữ liệu là só: 18
Số cột dữ liệu là phân loại: 5


In [51]:
# 06. Xử lý Outlier cho các cột số 
# Clip giá trị trong khoảng percentile 1% - 99% để loại bỏ nhiễu cực đoan
lower_limit = np.percentile(X_num, 1, axis=0)
upper_limit = np.percentile(X_num, 99, axis=0)
# NumPy Broadcasting so sánh và clip toàn bộ ma trận cùng lúc
X_num_clipped = np.clip(X_num, lower_limit, upper_limit)
print("Đã xử lý Outlier xong.")

Đã xử lý Outlier xong.


In [52]:
# 07. One-Hot Encoding (Vectorized Implementation)
onehot_list = []
onehot_header_list = []
# Duyệt qua từng cột category
for i, col_name in enumerate(cat_header):
    col_data = X_cat[:, i]
    unique_vals = np.unique(col_data)
    # Tạo ma trận one-hot bằng broadcasting
    onehot_matrix = (col_data[:, None] == unique_vals).astype(int)
    onehot_list.append(onehot_matrix)
    onehot_header_list.extend([f"{col_name}_{val}" for val in unique_vals])

X_cat_encoded = np.hstack(onehot_list)
# Ghép Numeric (đã clip) và Categorical (đã encode)
X_final = np.hstack((X_num_clipped, X_cat_encoded))
final_header = np.concatenate((num_header, onehot_header_list))
print("Final shape:", X_final.shape)

Final shape: (10127, 38)


In [53]:
# 08. Lưu file processed (Vectorized Save)
out_csv = "../data/processed/BankChurners_processed.csv"
# Gộp X và y để lưu; y đang là vector (N,), cần reshape thành (N, 1) để ghép
data_to_save = np.hstack((X_final, y.reshape(-1, 1)))
# Tạo header string
header_str = ",".join(list(final_header) + ["target"])
np.savetxt(out_csv, data_to_save, delimiter=",", header=header_str, comments="", fmt='%.6f')
print(f"Đã lưu dữ liệu đã xử lý tại: {out_csv}")

Đã lưu dữ liệu đã xử lý tại: ../data/processed/BankChurners_processed.csv


## Tổng kết và Nhận xét:
Sau quá trình tiền xử lý dữ liệu sử dụng hoàn toàn thư viện NumPy, em đã chuyển đổi dữ liệu thô thành dạng vector số học sạch sẽ, sẵn sàng cho mô hình học máy.

1. Tối ưu hóa hiệu năng (NumPy Vectorization)

Không sử dụng vòng lặp (No For-Loops): Toàn bộ quá trình từ điền giá trị thiếu, chuẩn hóa đến One-Hot Encoding đều sử dụng cơ chế Vectorization và Broadcasting của NumPy. Điều này giúp giảm thời gian xử lý xuống đáng kể so với việc dùng Python List thuần, đồng thời code gọn gàng và dễ bảo trì hơn.

2. Kỹ thuật xử lý dữ liệu

- Missing Values: Đã xử lý các giá trị "Unknown" bằng phương pháp Mode Imputation (điền giá trị xuất hiện nhiều nhất) để giữ nguyên phân phối của dữ liệu phân loại.

- Outlier Handling: Áp dụng kỹ thuật Winsorization (kẹp giá trị trong khoảng percentile 1% - 99%) cho các cột số. Việc này giúp hạn chế tác động tiêu cực của các giá trị ngoại lai (ví dụ: hạn mức tín dụng quá cao) lên các mô hình tuyến tính sau này.

3. Tạo đặc trưng mới

Em đã tạo thêm 2 đặc trưng mới dựa trên kiến thức tham khảo:

- Avg_Trans_Amt_Per_Transaction: Tỷ lệ giữa Tổng tiền và Tổng số lần giao dịch. Biến này giúp mô hình phân biệt được nhóm khách hàng "giao dịch ít nhưng tiền lớn" và "giao dịch lặt vặt".

- Credit_Utilization_Rate_Calc: Tính toán lại tỷ lệ sử dụng tín dụng chính xác từ số dư nợ và hạn mức, giúp củng cố thông tin cho mô hình.

4. Kết quả cuối cùng

- Dữ liệu đầu ra đã được mã hóa toàn bộ (One-Hot cho biến phân loại, Float cho biến số).

- Shape cuối cùng: (10127, 39) (bao gồm cả các cột One-Hot mới sinh ra).

- Dữ liệu đã được lưu tại ../data/processed/BankChurners_processed.csv, sẵn sàng cho bước Modeling.